# How to Access GES DISC Data Using Python

<p></p>

<div style="background:#eeeeee; border:1px solid #cccccc;padding:5px 10px;">Please, be very judicious when working on long data time series residing on a remote data server.<br />
It is very likely that attempts to apply similar approaches on remote data, such as hourly data, for more than a year of data at a time, will result in a heavy load on the remote data server. This may lead to negative consequences, ranging from very slow performance that will be experienced by hundreds of other users, up to denial of service.</div>

### Overview

There are multiple ways to work with GES DISC data resources using Python. For example, the data can accessed using [techniques that rely on a native Python code](https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html). 

Still, there are several third-party libraries that can further simplify the access. In the sections below, we demonstrate downloading and streaming granules to the notebook using these libraries.

The examples will use a sample MERRA-2 granule, from the [M2T1NXSLV.5.12.4 collection](https://disc.gsfc.nasa.gov/datasets/M2T1NXSLV_5.12.4/summary?keywords=M2T1NXSLV_5.12.4), to demonstrate data access.

### Prerequisites

This notebook was written using Python 3.10, and requires these libraries and files:

- `netrc` file with valid Earthdata Login credentials
   - [How to Generate Earthdata Prerequisite Files](https://disc.gsfc.nasa.gov/information/howto?title=How%20to%20Generate%20Earthdata%20Prerequisite%20Files)
- [requests](https://docs.python-requests.org/en/latest/) (version 2.22.0 or later)
- [pydap](https://github.com/pydap/pydap) (we recommend using version 3.4.0 or later)
- [xarray](https://docs.xarray.dev/en/stable/)
- [netCDF4-python](https://github.com/Unidata/netcdf4-python) (we recommend using version 1.6.2)
- [earthaccess](https://earthaccess.readthedocs.io/en/latest/quick-start/)

#### Optional Anaconda Environment YAML:

This notebook can be run using the ['opendap' YAML file](https://github.com/nasa/gesdisc-tutorials/tree/main/environments/opendap.yml) provided in the 'environments' subfolder.

Please follow the instructions [here](https://conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html#creating-an-environment-from-an-environment-yml-file) to install and activate this environment. 

### Links Used in this Notebook

There are several example links that will be used to access data from the same granule. Each link can be searched for using several tools, including [Earthdata Search](https://search.earthdata.nasa.gov/search/granules/granule-details?p=C1276812863-GES_DISC&pg[0][v]=f&pg[0][qt]=1980-01-01%2C1981&pg[0][gsk]=-start_date&g=G1277898447-GES_DISC&q=M2T1NXSLV&tl=1723660883!3!!), the [dataset landing page](https://disc.gsfc.nasa.gov/datasets/M2T1NXSLV_5.12.4/summary?keywords=M2T1NXSLV_5.12.4) for the particular collection, or through the [Content Metadata Repository](https://cmr.earthdata.nasa.gov/virtual-directory/collections/C1276812863-GES_DISC/temporal/1980/01/01).

Links used in this notebook:
- HTTPS: https://data.gesdisc.earthdata.nasa.gov/data/MERRA2/M2T1NXSLV.5.12.4/1980/01/MERRA2_100.tavg1_2d_slv_Nx.19800101.nc4
- OPeNDAP (for subsetting): 
    - DAP4 Subsetting Page: https://opendap.earthdata.nasa.gov/collections/C1276812863-GES_DISC/granules/M2T1NXSLV.5.12.4%3AMERRA2_100.tavg1_2d_slv_Nx.19800101.nc4.dmr.html
    - Example Subsetted URL: 

### 1. Download Full Granules Using Python

#### Option 1: Use `requests`

`requests` is a popular Python library that simplifies Python access to Internet-based resources. In the following code, we demonstrate how to use 'Requests' to access GES DISC data using cookies created by a host operating system.

In [ ]:
import requests

URL = 'https://data.gesdisc.earthdata.nasa.gov/data/MERRA2/M2T1NXSLV.5.12.4/1980/01/MERRA2_100.tavg1_2d_slv_Nx.19800101.nc4'

# Set the FILENAME string to the data file name, the LABEL keyword value, or any customized name. 
FILENAME = 'your_file_string_goes_here'

import requests
result = requests.get(URL)
try:
    result.raise_for_status()
    f = open(FILENAME,'wb')
    f.write(result.content)
    f.close()
    print('contents of URL written to '+FILENAME)
except:
    print('requests.get() returned an error code '+str(result.status_code))

#### Option 2: Use `earthaccess`

The `earthaccess` library can be used to search for granules and download them to your local machine. The `search_data` function will search for granules inside the specified temporal and bounding box ranges, and will return a list of URLs to be downloaded. Finally, it will download these URLs, assuming you have been authenticated using your previously-generated Earthdata prerequisite files.

In [ ]:
import earthaccess

# This will work if Earthdata prerequisite files have already been generated
auth = earthaccess.login()

# To download multiple files, change the second temporal parameter
results = earthaccess.search_data(
    short_name='M2T1NXSLV.5.12.4',
    temporal=('1980-01-01', '1980-01-02'), # This will download the same 1980-01-01 granule used elsewhere in this notebook
    bounding_box=(-180, 0, 180, 90)
)

downloaded_files = earthaccess.download(
    results,
    local_path='.', # Change this string to download to a different path
)

### 2. Subset and Stream Data From OPeNDAP Servers

#### Option 1: Use `pydap` with Earthdata Login Credentials

A convenient access to GES DISC OPeNDAP resources can be also achieved with 'Pydap', a Python library that both provides an interface for Python programs to read from OPeNDAP servers and the netCDF4 Python module which uses the netCDF-C library to actually access data.



In [ ]:
from pydap.client import open_url
from pydap.cas.urs import setup_session
import getpass

dataset_url = 'https://servername/opendap/path/file[.format[?subset]]'

prompts = [
    'Enter NASA Earthdata Login Username \n(or create an account at urs.earthdata.nasa.gov): ',
    'Enter NASA Earthdata Login Password: '
]

username = input(prompts[0])
password = getpass.getpass(prompts[1])

try:
    session = setup_session(username, password, check_url=dataset_url)
    dataset = open_url(dataset_url, session=session)
    dataset.tree() # Open the dataset and view its contents
except AttributeError as e:
    print('Error:', e)
    print('Please verify that the dataset URL points to an OPeNDAP server, the OPeNDAP server is accessible, or that your username and password are correct.')

### Python using 'Xarray'

If you wish to open datasets as Xarray data objects, you can simply pass in a dataset URL to the <code>open_dataset()</code> function. Depending on the subsetting service that you wish to access, different Earthdata authentication files may be required. Here, we demonstrate accessing a granule via OPeNDAP and THREDDS.

#### OPeNDAP in Xarray:


In [ ]:
import xarray as xr

# Reading a single granule URL:
ds = xr.open_dataset('https://servername/opendap/path/file[.format[?subset]]')

#### THREDDS in Xarray:

Datasets that include <code>.ncml</code> aggregation, like some provided through THREDDS, may be useful for quickly subsetting multiple granules into a single data array.

This operation requires a <code>.dodsrc</code> file in your root and working directories, and a <code>.netrc</code> file in your root directory.

In [ ]:
# Subsetting a .ncml file URL:
URL = 'https://servername/thredds/dodsC/path/dataset_Aggregation.ncml'

try:
    ds = xr.open_dataset(URL).sel(lat=slice(lat1,lat2),lon=slice(lon1,lon2),time=slice(time1,time2))
except OSError as e:
    print('Error', e)
    print('Please check that your .dodsrc files are in their correct locations, or that your .netrc file has the correct username and password.')

### Python using 'netCDF4-python'

netCDF4-python is a Python library that uses the [netCDF-c](https://github.com/Unidata/netcdf-c) library to open and read netCDF4 files. It can be used to remotely access OPeNDAP netCDF4 granules, or locally downloaded netCDF4 granules.

#### OPeNDAP in netCDF4-python:

This step requires a `.netrc` file in your root directory.

In [ ]:
import netCDF4 as nc4

nc = nc4.Dataset('https://servername/opendap/path/file[.nc4[?subset]]')